## Zooniverse Analysis

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import numpy as np
import seaborn as sns
from IPython.display import Image, display, HTML


In [ ]:
sns.set_context('poster')

In [ ]:
dfzoo = pd.read_csv('/epyc/users/ecbellm/ZTF_Boyajian/ztf-dippers-classifications.csv')

In [ ]:
len(dfzoo)

In [ ]:
dfzoo.head()

this is one classification per row.

In [ ]:
dfzoo.iloc[0]['annotations']

we only have followup questions for the yes and maybe classificaitons, and some of the early yes/maybe classifications don't have followup at all

In [ ]:
dfzoo.loc[2444]['annotations']

In [ ]:
dfzoo[['classification_id','user_name']].groupby('user_name').agg(len).sort_values(by='classification_id')

In [ ]:
def get_classification(df):
    return dfzoo['annotations'].apply(json.loads).apply(lambda x: x[0]['value'])

def get_well_sampled(df):
    return dfzoo['annotations'].apply(json.loads).apply(lambda x: x[1]['value'] if (len(x) > 1) else None)

def get_ps1_id(df):
    return dfzoo['subject_data'].apply(json.loads).apply(lambda x: list(x.values())[0]['ps1_id'])

In [ ]:
dfzoo['classification'] = get_classification(dfzoo)
dfzoo['well_sampled'] = get_well_sampled(dfzoo)
dfzoo['ps1_id'] = get_ps1_id(dfzoo)

In [ ]:
wyes = dfzoo['classification'] == 'Yes'
wmaybe = dfzoo['classification'] == 'Maybe'
wyesmaybe = wyes | wmaybe
wno = ~wyesmaybe

In [ ]:
dfzoo.loc[wyes,'well_sampled']

In [ ]:
np.sum(wyes)

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(data=dfzoo,y='classification')
#plt.savefig(f'fig/{dataset}/classification_summary.png', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(data=dfzoo.loc[wyes],y='user_name')

In [ ]:
dfzoo.groupby('ps1_id').agg(len)['classification_id'].index[935]

In [ ]:
wmax_id = dfzoo['ps1_id'] == 172521268464107338
dfzoo.loc[wmax_id,['user_name','classification']]

In [ ]:
!ls

In [ ]:
image_dir = '/epyc/users/kyboone/ztf_boyajian/zooniverse/'

In [ ]:
display(Image(f'{image_dir}/172521268464107338.png'))

## Top tier: at least two yes votes
(still to do: check for "well-sampled" value, if we think it's needed)

In [ ]:
wbest = wyes #& (dfzoo['user_name'] != 'kboone')
count_yes = dfzoo[['ps1_id','classification']].loc[wbest].groupby('ps1_id').agg(len)
wtwoyes = count_yes['classification'] > 1
best_ids = count_yes.loc[wtwoyes].index.values


In [ ]:
len(best_ids)

In [ ]:
print(best_ids)

In [ ]:
for idi in best_ids:
    display(Image(f'{image_dir}/{idi}.png'))

### Second tier: only one yes (> 2 yes is above), at least one maybe

In [ ]:
count_yes = dfzoo[['ps1_id','classification']].loc[wyes].groupby('ps1_id').agg(len)
count_yes = count_yes.loc[count_yes['classification'] == 1]
count_maybe = dfzoo[['ps1_id','classification']].loc[wmaybe].groupby('ps1_id').agg(len)

count_yes_and_maybe = count_yes.join(count_maybe,lsuffix='_yes',rsuffix='_maybe',how='inner')

In [ ]:
okay_ids = count_yes_and_maybe.index.values

In [ ]:
len(okay_ids)

In [ ]:
print(okay_ids)

In [ ]:
for idi in okay_ids:
    display(Image(f'{image_dir}/{idi}.png'))